In [8]:
# imports and dependencies, api_key
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import gmaps

In [9]:
# import gmaps API key for heat map
#from config import gkey
#
## Configure gmaps
#gmaps.configure(api_key=gkey)

In [10]:
# file to load
path = "./Accident_Info.csv"

In [11]:
# initial data read for San Francisco
data = pd.read_csv( path )

In [12]:
# Clean data of duplicates and missing values, rename columns
df = pd.DataFrame( data )

In [13]:
# formatting of data, NaN handling
df = df[["Accident_Index", "Accident_Severity",
         "Date", "Day_of_Week",
         "Junction_Control", "Junction_Detail",
         "Latitude", "Longitude", "Light_Conditions",
         "Road_Surface_Conditions", "Speed_limit",
         "Time", "Urban_or_Rural_Area",
         "Weather_Conditions", "Year"]]
df = df.rename(index=str, columns={"Accident_Index": "Accident",
                                   "Accident_Severity": "Severity",
                                   "Day_of_Week": "Day",
                                   "Junction_Control": "Traffic Control",
                                   "Junction_Detail": "Junction Type",
                                   "Light_Conditions": "Light",
                                   "Road_Surface_Conditions": "Road Cond.",
                                   "Speed_limit": "Speed Limit",
                                   "Urban_or_Rural_Area": "City Type",
                                   "Weather_Conditions": "Weather"})
df = df.set_index( "Accident" )

In [14]:
# preview of data
df.head()

,Severity,Date,Day,Traffic Control,Junction Type,Latitude,Longitude,Light,Road Cond.,Speed Limit,Time,City Type,Weather,Year
Accident,,,,,,,,,,,,,,
200501BS00001,Serious,1/4/2005,Tuesday,Data missing or out of range,Not at junction or within 20 metres,51.489096,-0.191170,Daylight,Wet or damp,30,17:42,Urban,Raining no high winds,2005
200501BS00002,Slight,1/5/2005,Wednesday,Auto traffic signal,Crossroads,51.520075,-0.211708,Darkness - lights lit,Dry,30,17:36,Urban,Fine no high winds,2005
200501BS00003,Slight,1/6/2005,Thursday,Data missing or out of range,Not at junction or within 20 metres,51.525301,-0.206458,Darkness - lights lit,Dry,30,0:15,Urban,Fine no high winds,2005
200501BS00004,Slight,1/7/2005,Friday,Data missing or out of range,Not at junction or within 20 metres,51.482442,-0.173862,Daylight,Dry,30,10:35,Urban,Fine no high winds,2005
200501BS00005,Slight,1/10/2005,Monday,Data missing or out of range,Not at junction or within 20 metres,51.495752,-0.156618,Darkness - lighting unknown,Wet or damp,30,21:13,Urban,Fine no high winds,2005


## Location and Time Trends

### How do location and time of day affect severity of accidents? [Tim]

Does the location within London have an affect on the severity of an accident?

In [ ]:
# pull above data - CHANGE data REFERENCE ( don't rewrite master copy of data )

In [ ]:
# re-format/style data for trend1

In [ ]:
# manipulate data
# setup analysis

In [ ]:
# preview of data

In [ ]:
# visualization of data
# consider the best visualization for what you want to show

In [ ]:
# free space

Are accidents more or less severe on a particular day of the week?

In [ ]:
# re-format/style data for trend2

In [ ]:
# manipulate data
# setup analysis

In [ ]:
# preview of data

In [ ]:
# visualization of data
# consider the best visualization for what you want to show

In [ ]:
# free space

Does the time of day play a role in the number of severe accidents?

In [ ]:
# re-format/style data for trend3

In [ ]:
# manipulate data
# setup analysis

In [ ]:
# preview of data

In [ ]:
# visualization of data
# consider the best visualization for what you want to show

Overall, it appears that _ _ _ _ _ _ _ _ _ _ _ _ 

## Weather Conditions

### How do weather and visibility affect accident trends?

trend1 description / question / prompt

In [ ]:
# pull above data - CHANGE data REFERENCE ( don't rewrite master copy of data )

In [ ]:
# re-format/style data for trend1

In [ ]:
# manipulate data
# setup analysis

In [ ]:
# preview of data

In [ ]:
# visualization of data
# consider the best visualization for what you want to show

In [ ]:
# free space

trend2 description / question / prompt

In [ ]:
# re-format/style data for trend2

In [ ]:
# manipulate data
# setup analysis

In [ ]:
# preview of data

In [ ]:
# visualization of data
# consider the best visualization for what you want to show

In [ ]:
# free space

trend3 description / question / prompt

In [ ]:
# re-format/style data for trend3

In [ ]:
# manipulate data
# setup analysis

In [ ]:
# preview of data

In [ ]:
# visualization of data
# consider the best visualization for what you want to show

Any comments on observations?

## Subject Headline

### Subject focus

trend1 description / question / prompt

In [ ]:
# pull above data - CHANGE data REFERENCE ( don't rewrite master copy of data )

In [ ]:
# re-format/style data for trend1

In [ ]:
# manipulate data
# setup analysis

In [ ]:
# preview of data

In [ ]:
# visualization of data
# consider the best visualization for what you want to show

In [ ]:
# free space

trend2 description / question / prompt

In [ ]:
# re-format/style data for trend2

In [ ]:
# manipulate data
# setup analysis

In [ ]:
# preview of data

In [ ]:
# visualization of data
# consider the best visualization for what you want to show

In [ ]:
# free space

trend3 description / question / prompt

In [ ]:
# re-format/style data for trend3

In [ ]:
# manipulate data
# setup analysis

In [ ]:
# preview of data

In [ ]:
# visualization of data
# consider the best visualization for what you want to show

Any comments on observations?